In [ ]:
# Importing all necessary libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224


In [ ]:
directory="path/to/data"
count=0
for name in os.listdir(directory):
    count+=len([f for f in os.listdir(os.path.join(directory,name)) if os.path.isfile(os.path.join(os.path.join(directory,name), f))])
print(count)

In [ ]:
train_data_dir = "path/to/data"
validation_data_dir = "path/to/data"
nb_train_samples =count
nb_validation_samples = count
epochs = 20
batch_size = 5


In [ ]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)


In [ ]:
model = Sequential()
model.add(Conv2D(32, (4, 4), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
model.save('model_saved4.h5')


In [ ]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

from keras.models import load_model

model = load_model('model_saved3.h5')

image = load_img("path/to/data (3)/rice_plant_lacks_nutrients/Nitrogen(N)/untitled-94.JPG", target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
maxIndex=-1
a=label[0]
i=0
import numpy as np
i=np.argmax(a)
if i==0:
    print("class is potassium")
elif i==1:
    print("class is phosphorous")
else:
    print("class is nitrogen")
#print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])
print(label)

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import load_img
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
doc=""
model = load_model('model_saved4.h5')
def predict_image(img):
    
    image=load_img(img,target_size=(224,224))
    img1 = np.array(image)
    img1 = img1 / 255.0
    img1 = img1.reshape(1,224,224,3)
    label = model.predict(img1)
    a=label[0]
    i=0
    i=np.argmax(a)
    if i==0:
     return "1"
    elif i==1:
     return "2"
    elif i==2:
     return "3"
    elif i==3:
     return "4"
    elif i==4:
     return "5"
    elif i==5:
     return "6"
    else :
     return "7"
    

In [ ]:
import os
import sys
from langchain.embeddings import AzureOpenAIEmbeddings,OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
import constants
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter

from langchain.document_loaders import PyPDFLoader
def botmessage(query,cl):
 doc=""
 if cl=="3":
     doc="path/to/data/docs/Phosporous deficiency in rice.pdf"
 elif cl=="4":
     doc="path/to/data/docs/Potasium deficiency in rice.pdf"
 #elif cl=="3":
     doc="path/to/data/docs/Nitrogen deficiency in rice.pdf"
 elif cl=="2":
     doc="path/to/data/docs/Apple_Black_rot.pdf"
 #elif cl=="5":
     doc="path/to/data/docs/Apple_Cedar_rust.pdf"
 elif cl=="1":
     doc="path/to/data/docs/apple_scab.pdf"
 loader = PyPDFLoader(doc)
 documents=loader.load()
 text_splitter=CharacterTextSplitter(chunk_size=3000,chunk_overlap=0)
 texts=text_splitter.split_documents(documents)
 embeddings=OpenAIEmbeddings()
 search=Chroma.from_documents(texts,embeddings)
 chain=RetrievalQA.from_chain_type(llm=ChatOpenAI(),chain_type='stuff',retriever=search.as_retriever())
 return chain.run(query)


In [ ]:
import gradio as gr
import time
with gr.Blocks() as demo:
 image = gr.Image(type= 'filepath',height=300,width=300)
 msg = gr.Textbox(visible=False)
 #submit_button1 = gr.Button("Submit")
 #submit_button1.click(fn=predict_image,inputs=image,outputs=image)
 gr.Interface(fn=predict_image,inputs=image,outputs=msg)
 chatbot = gr.Chatbot()
 
 
 msg2=gr.Textbox()
 #submit_button2 = gr.Button("Submit")
 clear = gr.ClearButton([msg2, chatbot])
 def respond(message,cl, chat_history):
        bot_message = botmessage(query=message,cl=cl)
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history
 
 msg2.submit(respond, [msg2,msg, chatbot], [msg2, chatbot])
 
 #image.submit(respond)
if __name__=="__main__":
  demo.launch()

In [ ]:
predict_image("path/to/data/apple/Apple___Black_rot/ff4e792b-14c5-41db-af9e-0676abf8d20a___JR_FrgE.S 3088_new30degFlipLR.JPG")
